## Version 4
Getting started with MLflow.

Add autoreload since we are now modifying scripts in other folders.

In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
from dotenv import load_dotenv
load_dotenv("/home/lkwasniewski/repos/00001_Studia/MLOPSgradedFinal/.env", override=True)


False

In [92]:
import os
print(os.getenv("AWS_ACCESS_KEY"))  # Should print the key


None


In [93]:
from dotenv import dotenv_values
from pathlib import Path

# Load .env using expanded path
env_path = Path.home() / "repos/00001_Studia/MLOPSgradedFinal/.env"
print(f"Loading environment variables from: {env_path}")
env_vars = dotenv_values(dotenv_path=env_path)  # returns a dict

# Print all loaded env vars
for key, value in env_vars.items():
    print(f"{key} = {value}")


Loading environment variables from: /home/lkwasniewski/repos/00001_Studia/MLOPSgradedFinal/.env


Getting the data.

In [94]:

import os
from pathlib import Path
container_check = os.getenv("iscontainer")
if container_check=="y":
    config_dir = Path("/home/vscode/.config/kaggle")
    config_dir.mkdir(parents=True, exist_ok=True)

    with open(config_dir / "kaggle.json", "w") as dst:
        with open("../kaggle.json", "r") as src:
            dst.write(src.read())

In [95]:

import os
import sys

sys.path.append(os.path.abspath(".."))

from ARISA_DSML.config import RAW_DATA_DIR, categorical, target
from ARISA_DSML.preproc import get_raw_data


In [96]:
from ARISA_DSML.helpers import get_active_branch_name, get_git_commit_hash

get_active_branch_name("..")  # need ".." because we are not in root
get_git_commit_hash()


'0b0fb40275f40a8c6468555c34f752e10679be29'

In [97]:
get_raw_data()
!dir "{RAW_DATA_DIR}"

2025-06-20 21:53:08.159 | INFO     | ARISA_DSML.preproc:get_raw_data:19 - RAW_DATA_DIR is: /home/lkwasniewski/repos/00001_Studia/MLOPSGradedFinal/data/raw


Dataset URL: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
bots_vs_users.csv  diabetes_new_test.csv  test.csv
diabetes.csv	   diabetes_test.csv


Load preproc from preproc.py and run on train data:

In [98]:
from ARISA_DSML.preproc import preprocess_df

train_path = preprocess_df(RAW_DATA_DIR / "diabetes.csv")

/home/lkwasniewski/repos/00001_Studia/MLOPSGradedFinal/ARISA_DSML/preproc.py:33: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/home/lkwasniewski/repos/00001_Studia/MLOPSGradedFinal/ARISA_DSML/preproc.py:33: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/home/lkwasniewski/repos/00001_Studia/MLOPSGradedFinal/ARISA_DSML/preproc.py:33: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option

In [99]:
train_path

PosixPath('/home/lkwasniewski/repos/00001_Studia/MLOPSGradedFinal/data/processed/diabetes.csv')

In [100]:
import pandas as pd

df_train = pd.read_csv(train_path)
df_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,125.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,125.0,26.6,0.351,31,0
2,8,183.0,64.0,29.0,125.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [101]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    float64
 2   BloodPressure             768 non-null    float64
 3   SkinThickness             768 non-null    float64
 4   Insulin                   768 non-null    float64
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(6), int64(3)
memory usage: 54.1 KB


Split into X and y, get categorical indices:

In [102]:
print(df_train.columns.tolist())

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']


In [103]:
y_train = df_train.pop(target)
X_train = df_train

#categorical_indices = [X_train.columns.get_loc(col) for col in categorical if col in X_train.columns]
cacategorical = []
categorical_indices = []

And run the hyperparameter optimization (if ):

In [104]:
import mlflow
from ARISA_DSML.train import run_hyperopt, get_or_create_experiment

mlflow.set_tracking_uri(f"http://13.49.221.214:5000/")

experiment_id = get_or_create_experiment("diabetes_hyperparam_tuning")
mlflow.set_experiment(experiment_id=experiment_id)
best_params_path = run_hyperopt(X_train, y_train, categorical_indices)


2025-06-20 21:53:10.119 | INFO     | ARISA_DSML.train:run_hyperopt:82 - Best Parameters: {'depth': 3, 'learning_rate': 0.2847856453953337, 'iterations': 101, 'l2_leaf_reg': 0.08064725446361332, 'bagging_temperature': 0.4441828029429478, 'random_strength': 9.931197047043607}


In [105]:
print([X_train.columns[i] for i in categorical_indices])

[]


Do cross validation again:

In [106]:
import joblib
from ARISA_DSML.train import train_cv
params = joblib.load(best_params_path)
n_folds = 5
cv_output_path = train_cv(X_train, y_train, categorical_indices, params, n=n_folds)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 3.23ms	remaining: 323ms
1:	learn: 0.5061728	test: 0.5714286	best: 0.5714286 (1)	total: 7.58ms	remaining: 375ms
2:	learn: 0.4728435	test: 0.5185185	best: 0.5714286 (1)	total: 14.1ms	remaining: 460ms
3:	learn: 0.4300341	test: 0.5063291	best: 0.5714286 (1)	total: 19.1ms	remaining: 463ms
4:	learn: 0.4774194	test: 0.5128205	best: 0.5714286 (1)	total: 23.1ms	remaining: 443ms
5:	learn: 0.4922118	test: 0.5250000	best: 0.5714286 (1)	total: 26.5ms	remaining: 420ms
6:	learn: 0.5498489	test: 0.5679012	best: 0.5714286 (1)	total: 30ms	remaining: 403ms
7:	learn: 0.5481928	test: 0.5679012	best: 0.5714286 (1)	total: 33.8ms	remaining: 393ms
8:	learn: 0.5552239	test: 0.5609756	best: 0.5714286 (1)	total: 35.7ms	remaining: 365ms
9:	learn: 0.5568862	test: 0.5609756	best: 0.5714286 (1)	total: 37.9ms	remaining: 345ms
10:	learn: 0.5780347	test: 0.5882353	best: 0.5882353 (10)	total: 39.7ms	remaining: 325ms
11:	learn: 0.5786517

And get the performance plots:

In [107]:
from ARISA_DSML.train import plot_error_scatter
cv_results = pd.read_csv(cv_output_path)
plot_error_scatter(
    df_plot=cv_results, 
    name="Mean F1 Score", 
    title="Cross-Validation (N=5) Mean F1 score with Error Bands",
    xtitle="Training Steps",
    ytitle="Performance Score",
    yaxis_range=[0.5, 1]
)

In [108]:
plot_error_scatter(
    cv_results,
    x="iterations",
    y="test-Logloss-mean",
    err="test-Logloss-std",
    name="Mean logloss",
    title="Cross-Validation (N=5) Mean Logloss with Error Bands",
    xtitle="Training Steps",
    ytitle="Logloss",
)


Fit model on full dataset:

In [109]:
from ARISA_DSML.train import train
params = joblib.load(best_params_path)
experiment_id = get_or_create_experiment("diabetes_hyperparam_tuning")
mlflow.set_experiment(experiment_id=experiment_id)
cv_results = pd.read_csv(cv_output_path)
model_path, model_params_path = train(X_train, y_train, categorical_indices, params, cv_results=cv_results)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6612246	total: 1.64ms	remaining: 164ms
50:	learn: 0.3681974	total: 31.5ms	remaining: 30.9ms
100:	learn: 0.2604910	total: 45.7ms	remaining: 0us
🏃 View run languid-boar-927 at: http://13.49.221.214:5000/#/experiments/2/runs/f3d91c96c25b4fdf8c2179b7620cd662
🧪 View experiment at: http://13.49.221.214:5000/#/experiments/2


S3UploadFailedError: Failed to upload /home/lkwasniewski/repos/00001_Studia/MLOPSGradedFinal/models/catboost_model_diabetes.cbm to mlflow-artifacts-prod/2/f3d91c96c25b4fdf8c2179b7620cd662/artifacts/catboost_model_diabetes.cbm: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied

And now for the test set, because of our refactor, we don't need to copy paste anymore:

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

In [ ]:
import os
print(os.getenv("ARTIFACT_ROOT"))  # Should print your-access-key

In [ ]:
dataset_test_path = RAW_DATA_DIR / "test.csv"
preprocced_test_path = preprocess_df(dataset_test_path)
df_test = pd.read_csv(preprocced_test_path)
df_test["Age"].unique()

df_test.head()

In [ ]:
df_train.head()

Get predictions and shap plot:

In [ ]:
import os
import json
import pandas as pd
import mlflow
from mlflow.client import MlflowClient
import joblib
from ARISA_DSML.predict import predict
from ARISA_DSML.config import PROCESSED_DATA_DIR, MODELS_DIR

# Load diabetes test set
df_test = pd.read_csv(PROCESSED_DATA_DIR / "diabetes.csv")

# Initialize MLflow client and get latest model info
client = MlflowClient(mlflow.get_tracking_uri())
model_name = "diabetes-binary-class"
model_info = client.get_latest_versions(model_name)[0]
run = client.get_run(model_info.run_id)

# Load model
_, artifact_folder = os.path.split(model_info.source)
model_uri = f"runs:/{model_info.run_id}/{artifact_folder}"
loaded_model = mlflow.catboost.load_model(model_uri)

# Load run parameters
run_data_dict = run.data.to_dictionary()
params = run_data_dict.get("params", {})

# Try to extract feature_columns from signature or fallback to model_params.pkl
try:
    log_model_meta = json.loads(run.data.tags["mlflow.log-model.history"])
    signature = log_model_meta[0].get("signature", None)
    if signature:
        params["feature_columns"] = [inp["name"] for inp in json.loads(signature["inputs"])]
        print("✅ Loaded feature_columns from model signature.")
except Exception as e:
    print(f"⚠️ Signature not found or invalid: {e}")

# Fallback: load model_params.pkl if feature_columns is still missing
if "feature_columns" not in params:
    try:
        model_params_path = MODELS_DIR / "model_params.pkl"
        if model_params_path.exists():
            params = joblib.load(model_params_path)
            print("✅ Loaded feature_columns from model_params.pkl.")
    except Exception as e:
        raise RuntimeError("❌ Cannot resolve feature_columns for prediction.") from e

# Run prediction
preds_path = predict(loaded_model, df_test, params, probs=True)
print(f"✅ Predictions saved to {preds_path}")


And that's it for the Mlflow demo.  

In [ ]:
from ARISA_DSML.resolve import get_model_by_alias
client = MlflowClient(mlflow.get_tracking_uri())
champ_mv = get_model_by_alias(client)
chall_mv = get_model_by_alias(client, alias="challenger")

In [ ]:
from ARISA_DSML.config import MODEL_NAME
client = MlflowClient(mlflow.get_tracking_uri())
model_info = get_model_by_alias(client, alias="champion")
if model_info is None:
    print("No champion model, predicting using newest model")
    model_info = client.get_latest_versions(MODEL_NAME)[0]

# extract params/metrics data for run `test_run_id` in a single dict 
run_data_dict = client.get_run(model_info.run_id).data.to_dictionary()
run = client.get_run(model_info.run_id)
log_model_meta = json.loads(run.data.tags['mlflow.log-model.history'])
log_model_meta[0]['signature']


_, artifact_folder = os.path.split(model_info.source)
model_uri = "runs:/{}/{}".format(model_info.run_id, artifact_folder)
print(model_uri)
loaded_model = mlflow.catboost.load_model(model_uri)

params = run_data_dict["params"]
params["feature_columns"] = [inp["name"] for inp in json.loads(log_model_meta[0]['signature']['inputs'])]
preds_path = predict(loaded_model, df_test, params)
